이전 코드를 gpu 상황에서 실행해보면서 바꿔야 할 부분을 확인해봅시다

* 우선 코랩의 런타임 유형을 gpu로 변경해봅시다. 

* 모델과, 데이터를 GPU로 옮겨야 합니다


In [1]:
import pandas as pd
import torch
from torch import nn

In [2]:
torch.cuda.is_available()   # gpu가 사용가능한지 확인

True

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Teaching/DL 202301/HousingData.csv')

In [4]:
df = df.dropna()  

In [5]:
x_df = df.iloc[:,:-1]
y_df = df.iloc[:,-1]

In [6]:
x = torch.from_numpy(x_df.values).float()
y = torch.from_numpy(y_df.values).float()

In [7]:
x = x.to('cuda')
y = y.to('cuda')

In [8]:
class DNN(nn.Module): # nn.Module subclassing

  def __init__(self):
    super().__init__() # base class 초기화
    self.fc1 = nn.Linear(13, 64) # 레이어 인스턴스 만들기
    self.relu = nn.ReLU() 
    self.fc2 = nn.Linear(64,1)

  def forward(self, x): # 데이터 입력 받을수 있게 변수 추가
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x


In [9]:
model = DNN() # 인스턴스 만들기

In [10]:
#
model = model.to('cuda')

In [11]:
model(x[:10])

tensor([[-0.8759],
        [ 0.1344],
        [-1.0050],
        [-2.9326],
        [-1.6331],
        [ 2.5441],
        [ 2.5149],
        [ 2.3795],
        [ 1.5319],
        [-3.4504]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [12]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
epochs = 1000
batch_size = 10

In [14]:
import time
start = time.time()
for epoch in range(epochs):
  running_loss = 0 # 전체 데이터셋에 대한 loss 계산

  # 데이터를 batch로 나누어서 학습
  for i in range(0, x.shape[0], batch_size): #range(start, end, step), end는 미포함
    inputs = x[i:i+batch_size]
    labels = y[i:i+batch_size]

    optimizer.zero_grad() # gradient 초기화
    outputs = model(inputs) # 모델값 예측
    #print(outputs)
    loss = loss_fn(outputs, labels.unsqueeze(dim=1)) # 에러 계산
    loss.backward() # gradient 계산
    optimizer.step() # gradient 업데이트

    running_loss += loss.item() * batch_size

  if (epoch+1)%10==0:   
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, running_loss / x.shape[0]))

print(time.time()-start)

Epoch [10/1000], Loss: 65.2045
Epoch [20/1000], Loss: 60.9021
Epoch [30/1000], Loss: 56.9219
Epoch [40/1000], Loss: 53.2326
Epoch [50/1000], Loss: 49.0519
Epoch [60/1000], Loss: 45.8528
Epoch [70/1000], Loss: 42.9729
Epoch [80/1000], Loss: 40.4174
Epoch [90/1000], Loss: 38.5225
Epoch [100/1000], Loss: 36.7936
Epoch [110/1000], Loss: 35.3262
Epoch [120/1000], Loss: 34.0755
Epoch [130/1000], Loss: 32.7265
Epoch [140/1000], Loss: 31.5321
Epoch [150/1000], Loss: 30.8759
Epoch [160/1000], Loss: 29.9703
Epoch [170/1000], Loss: 29.0993
Epoch [180/1000], Loss: 27.4034
Epoch [190/1000], Loss: 26.5045
Epoch [200/1000], Loss: 25.7456
Epoch [210/1000], Loss: 24.6990
Epoch [220/1000], Loss: 23.8428
Epoch [230/1000], Loss: 24.2875
Epoch [240/1000], Loss: 22.9567
Epoch [250/1000], Loss: 22.3544
Epoch [260/1000], Loss: 21.9388
Epoch [270/1000], Loss: 20.5599
Epoch [280/1000], Loss: 20.2414
Epoch [290/1000], Loss: 19.8129
Epoch [300/1000], Loss: 20.8145
Epoch [310/1000], Loss: 19.4053
Epoch [320/1000],

cpu로 학습하는 것과 시간을 비교해봅시다. 

In [15]:
x = x.to('cpu')
y = y.to('cpu')
model = DNN()

In [16]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 모델 파라미터가 입력으로 들어가므로 새로 인스턴스 만들어줘야함

In [17]:
start = time.time()
for epoch in range(epochs):
  running_loss = 0 # 전체 데이터셋에 대한 loss 계산

  # 데이터를 batch로 나누어서 학습
  for i in range(0, x.shape[0], batch_size): #range(start, end, step), end는 미포함
    inputs = x[i:i+batch_size]
    labels = y[i:i+batch_size]

    optimizer.zero_grad() # gradient 초기화
    outputs = model(inputs) # 모델값 예측
    #print(outputs)
    loss = loss_fn(outputs, labels.unsqueeze(dim=1)) # 에러 계산
    loss.backward() # gradient 계산
    optimizer.step() # gradient 업데이트

    running_loss += loss.item() * batch_size

  if (epoch+1)%10==0:   
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, running_loss / x.shape[0]))

print(time.time()-start)

Epoch [10/1000], Loss: 67.8809
Epoch [20/1000], Loss: 56.7580
Epoch [30/1000], Loss: 52.0138
Epoch [40/1000], Loss: 49.3299
Epoch [50/1000], Loss: 46.8493
Epoch [60/1000], Loss: 45.0197
Epoch [70/1000], Loss: 43.5450
Epoch [80/1000], Loss: 42.1054
Epoch [90/1000], Loss: 40.6551
Epoch [100/1000], Loss: 39.4830
Epoch [110/1000], Loss: 38.2429
Epoch [120/1000], Loss: 37.0825
Epoch [130/1000], Loss: 35.7412
Epoch [140/1000], Loss: 34.5331
Epoch [150/1000], Loss: 33.2891
Epoch [160/1000], Loss: 31.9344
Epoch [170/1000], Loss: 30.8822
Epoch [180/1000], Loss: 29.9060
Epoch [190/1000], Loss: 28.9092
Epoch [200/1000], Loss: 27.9487
Epoch [210/1000], Loss: 26.4322
Epoch [220/1000], Loss: 25.7499
Epoch [230/1000], Loss: 24.9168
Epoch [240/1000], Loss: 23.6848
Epoch [250/1000], Loss: 22.6376
Epoch [260/1000], Loss: 21.7867
Epoch [270/1000], Loss: 21.4795
Epoch [280/1000], Loss: 20.4534
Epoch [290/1000], Loss: 20.3323
Epoch [300/1000], Loss: 19.3792
Epoch [310/1000], Loss: 19.4103
Epoch [320/1000],

일반적으로 GPU 학습이 빠릅니다. 위의 예제는 작은 데이터셋을 학습하여서 GPU 사용 시 더 느리게 나왔습니다